In [32]:
import requests as re
res = re.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-11-01").json()

In [33]:
res

{'type': 'FeatureCollection',
 'metadata': {'generated': 1732617583000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-11-01',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.14.1',
  'count': 6272},
 'features': [{'type': 'Feature',
   'properties': {'mag': 0.69,
    'place': '10 km WNW of Cobb, CA',
    'time': 1732616961900,
    'updated': 1732617056350,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75090581',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75090581&format=geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'automatic',
    'tsunami': 0,
    'sig': 7,
    'net': 'nc',
    'code': '75090581',
    'ids': ',nc75090581,',
    'sources': ',nc,',
    'types': ',nearby-cities,origin,phase-data,',
    'nst': 9,
    'dmin': 0.007217,
    'rms': 0.02,
    'gap': 123,
    'magType': 'md',
    'type': 'earthquake',
    'tit

In [34]:
len(res["features"])

6272

In [13]:
from datetime import datetime, timezone
datetime.utcfromtimestamp(1732492753770/1000).strftime('%Y-%m-%dT%H:%M:%S')

'2024-11-24T23:59:13'

In [14]:
datetime.fromtimestamp(1732492753770 / 1000, timezone.utc).strftime("%Y-%m-%dT%H:%M:%S")

'2024-11-24T23:59:13'

In [38]:
datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

'2024-11-25T12:23:33'

In [44]:
import pymongo
def connect_mongo(
        mongo_client: str = "mongodb://localhost:27017/",
        mongo_db: str = "earthquake_db",
        mongo_collection: str = "earthquakes",
    ) -> tuple:
    """_summary_

    Parameters
    ----------
    mongo_client : _type_, optional
        url de connexion à la base de données MongoDB, by default "mongodb://localhost:27017/"
    mongo_db : str, optional
        _description_, by default "earthquake_db"
    mongo_collection : str, optional
        _description_, by default "earthquakes"

    Returns
    -------
    tuple
        Tuple contenant le client mongo, la db et la collection pour requêter la base de données
    """

    client = pymongo.MongoClient(mongo_client)
    db = client[mongo_db]
    collection = db[mongo_collection]

    return client, db, collection

In [46]:
client, db, collection = connect_mongo(mongo_db="superheroes", mongo_collection="info")

In [47]:
type(collection)

pymongo.synchronous.collection.Collection

In [69]:
type(client)

pymongo.synchronous.mongo_client.MongoClient

In [49]:
result = collection.aggregate([
    {"$group": {"_id": None, "maxScore": {"$max": "$weight(kg)"}}}
])

In [51]:
max_value = next(result, None)
if max_value:
    print("Valeur maximale :", max_value["maxScore"])
else:
    print("Aucun document trouvé.")

Valeur maximale : 900.0


In [ ]:
import pandas as pd

In [58]:
import pandas as pd
data=res["features"][0]
print(data)

{'type': 'Feature', 'properties': {'mag': 1.6, 'place': '53 km NW of Toyah, Texas', 'time': 1732530093219, 'updated': 1732530409152, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2024xdxb', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2024xdxb&format=geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'automatic', 'tsunami': 0, 'sig': 39, 'net': 'tx', 'code': '2024xdxb', 'ids': ',tx2024xdxb,', 'sources': ',tx,', 'types': ',origin,phase-data,', 'nst': 17, 'dmin': 0.1, 'rms': 0.5, 'gap': 103, 'magType': 'ml', 'type': 'earthquake', 'title': 'M 1.6 - 53 km NW of Toyah, Texas'}, 'geometry': {'type': 'Point', 'coordinates': [-104.18, 31.667, 5.5381]}, 'id': 'tx2024xdxb'}


In [61]:
res["features"][0]["properties"]

{'mag': 1.6,
 'place': '53 km NW of Toyah, Texas',
 'time': 1732530093219,
 'updated': 1732530409152,
 'tz': None,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2024xdxb',
 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2024xdxb&format=geojson',
 'felt': None,
 'cdi': None,
 'mmi': None,
 'alert': None,
 'status': 'automatic',
 'tsunami': 0,
 'sig': 39,
 'net': 'tx',
 'code': '2024xdxb',
 'ids': ',tx2024xdxb,',
 'sources': ',tx,',
 'types': ',origin,phase-data,',
 'nst': 17,
 'dmin': 0.1,
 'rms': 0.5,
 'gap': 103,
 'magType': 'ml',
 'type': 'earthquake',
 'title': 'M 1.6 - 53 km NW of Toyah, Texas'}

Au dessus, on veut garder ```mag```, ```place```, ```time```, ```type```, ```nst```, ```dmin```, ```sig```, ```magType```

In [66]:
res["features"][0]["geometry"]

{'type': 'Point', 'coordinates': [-104.18, 31.667, 5.5381]}

In [67]:
final_dict = {
    "mag": res["features"][0]["properties"]["mag"],
    "place": res["features"][0]["properties"]["place"],
    "time": res["features"][0]["properties"]["time"],
    "type": res["features"][0]["properties"]["type"],
    "nst": res["features"][0]["properties"]["nst"],
    "dmin": res["features"][0]["properties"]["dmin"],
    "sig": res["features"][0]["properties"]["sig"],
    "magType": res["features"][0]["properties"]["magType"],
    "geometryType" : res["features"][0]["geometry"]["type"],
    "coordinates" : res["features"][0]["geometry"]["coordinates"]
}

In [68]:
final_dict

{'mag': 1.6,
 'place': '53 km NW of Toyah, Texas',
 'time': 1732530093219,
 'type': 'earthquake',
 'nst': 17,
 'dmin': 0.1,
 'sig': 39,
 'magType': 'ml',
 'geometryType': 'Point',
 'coordinates': [-104.18, 31.667, 5.5381]}

In [27]:
import requests
import pymongo
def extract(
        client: str = "mongodb://localhost:27017/",
        db: str = "earthquake_db",
        collection: str = "earthquakes",
        request: str = "https://earthquake.usgs.gov/fdsnws/event/1/query?",
        format: str = "geojson",
        starttime: str = "2024-11-01",
    ) -> dict:
        """Tâche d'extraction de la donnée depuis l'API

        Parameters
        ----------
        request : _type_, optional
            corps de la requête à l'API, by default "https://earthquake.usgs.gov/fdsnws/event/1/query?"
        format : str, optional
            format de la réponse attendu, by default "geojson"
        starttime : str, optional
            paramètre de la requête, by default "2024-11-01"
        client : str
            client de la base de données, issu de la fonction connect_mongo()
        db : str
            nom de la base de données, issu de la fonction connect_mongo()
        collection : str
            nom de la collection, issu de la fonction connect_mongo()

        Returns
        -------
        dict
            dictionnaire contenant les résultats de la requête API
        """

        # -------------------------------------------------------------------------------------------------------------------------------------------#
        # Connexion à la base de données MongoDB
        client = pymongo.MongoClient(client)
        db = client[db]
        collection = db[collection]

        # -------------------------------------------------------------------------------------------------------------------------------------------#
        # Phase de récupération de la dernière date de requête

        res = collection.aggregate([{"$group": {"_id": None, "date": {"$max": "$date"}}}])
        

        # Si il n'y a aucun document, starttime est égal à celui prédéfini, sinon au max trouvé dans la collection MongoDB
        result = next(res, None)
        print(f"result : {result}")
        starttime = result["date"] if result else starttime
        print(f"starttime : {starttime}")

        # -------------------------------------------------------------------------------------------------------------------------------------------#
        # Requête API
        final_api_request = f"{request}format={format}&starttime={starttime}"

        raw_data = requests.get(final_api_request).json()

        # -------------------------------------------------------------------------------------------------------------------------------------------#
        # Fermeture du client MongoDB
        client.close()

        # -------------------------------------------------------------------------------------------------------------------------------------------#
        # Vérification de l'existence de features

        if len(raw_data["features"]) > 0:
            return raw_data
        else:
              print("Pas de nouvelles features")

In [28]:
extract

result : {'_id': None, 'date': '2024-11-26T09:21:32'}
starttime : 2024-11-26T09:21:32


{'type': 'FeatureCollection',
 'metadata': {'generated': 1732617069000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-11-26T09:21:32',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.14.1',
  'count': 5},
 'features': [{'type': 'Feature',
   'properties': {'mag': 0.69,
    'place': '10 km WNW of Cobb, CA',
    'time': 1732616961900,
    'updated': 1732617056350,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75090581',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75090581&format=geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'automatic',
    'tsunami': 0,
    'sig': 7,
    'net': 'nc',
    'code': '75090581',
    'ids': ',nc75090581,',
    'sources': ',nc,',
    'types': ',nearby-cities,origin,phase-data,',
    'nst': 9,
    'dmin': 0.007217,
    'rms': 0.02,
    'gap': 123,
    'magType': 'md',
    'type': 'earthquake',
  

In [40]:
import pendulum
pendulum.now('UTC').add(days=-1).strftime(
                    "%Y-%m-%dT%H:%M:%S"
                )

'2024-11-25T13:55:28'

In [43]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["earthquake_db"]
collection = db["earthquakes"]

yesterday = pendulum.now("UTC").add(days=-1).strftime("%Y-%m-%dT%H:%M:%S")

myquery = {"date": {"$lte": f"{yesterday}"}}

results = list(collection.find(myquery))

print(len(results))
for doc in results:
    print(doc)

6115
{'_id': ObjectId('674597b629fb05cd93029356'), 'mag': 4.4, 'place': '71 km WNW of San Antonio de los Cobres, Argentina', 'date': '2024-11-25T15:45:25', 'type': 'earthquake', 'nst': 22, 'dmin': 1.506, 'sig': 298, 'magType': 'mb', 'geometryType': 'Point', 'longitude': -66.9633, 'latitude': -23.9684, 'depth': 227.174}
{'_id': ObjectId('674597b629fb05cd93029357'), 'mag': 1.83, 'place': '10 km ESE of Pinnacles, CA', 'date': '2024-11-25T15:42:03', 'type': 'earthquake', 'nst': 15, 'dmin': 0.05308, 'sig': 52, 'magType': 'md', 'geometryType': 'Point', 'longitude': -121.039497375488, 'latitude': 36.5063323974609, 'depth': 7.78999996185303}
{'_id': ObjectId('674597b629fb05cd93029358'), 'mag': 5.2, 'place': 'Izu Islands, Japan region', 'date': '2024-11-25T15:37:42', 'type': 'earthquake', 'nst': 63, 'dmin': 2.427, 'sig': 416, 'magType': 'mb', 'geometryType': 'Point', 'longitude': 141.1387, 'latitude': 29.3494, 'depth': 115.461}
{'_id': ObjectId('674597b629fb05cd93029359'), 'mag': -0.44, 'place'

In [44]:
import requests
request = "https://nominatim.openstreetmap.org/reverse?lat=48.8566&lon=2.3522&format=json"
requests.get(request)

<Response [403]>

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")
location = geolocator.reverse((48.8566, 2.3522))  # Coordonnées de Paris

print(location.address)  # Adresse complète
print(location.raw["address"]["country"])  # Juste le pays